# Лаба 5


In [1]:
!pip install -q torch datasets==2.14.6 transformers evaluate sentencepiece accelerate rouge-score huggingface_hub



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
from __future__ import annotations

import sys
import os
import torch
from pathlib import Path

from dataclasses import dataclass
from typing import Optional

os.environ["WANDB_DISABLED"] = "true"
import pyarrow as pa

if not hasattr(pa, "PyExtensionType"):
    pa.PyExtensionType = pa.ExtensionType

to_drop = [k for k in list(sys.modules) if k == "datasets" or k.startswith("datasets.")]
for name in to_drop:
    del sys.modules[name]

from datasets import DatasetDict, load_dataset
from huggingface_hub import snapshot_download
from transformers import AutoTokenizer
from lab5_utils import train_task, generate_sample

print("torch", torch.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("dev", device)


torch 2.9.1+cu128
dev cuda


## data


In [3]:
MODEL_NAME = "ai-forever/ruT5-base"
DATASET_SIZES = {"train": "train[:1000]", "test": "test[:200]"}
OUTPUT_ROOT = Path("models")
OUTPUT_ROOT.mkdir(exist_ok=True)

def load_gazeta_dataset(
    train_slice: str = "train[:1000]",
    test_slice: str = "test[:200]",
    cache_dir: Optional[Path] = None,
) -> DatasetDict:
    repo_path = snapshot_download(
        repo_id="IlyaGusev/gazeta",
        revision="v2.0",
        repo_type="dataset",
        cache_dir=str(cache_dir) if cache_dir else None,
    )
    data_files = {
        "train": str(Path(repo_path) / "gazeta_train.jsonl"),
        "test": str(Path(repo_path) / "gazeta_test.jsonl"),
    }
    return load_dataset(
        "json",
        data_files=data_files,
        split={"train": train_slice, "test": test_slice},
        cache_dir=str(cache_dir) if cache_dir else None,
    )

raw_dataset = load_gazeta_dataset(
    train_slice=DATASET_SIZES["train"],
    test_slice=DATASET_SIZES["test"],
)
print("ds", raw_dataset)

DEFAULT_MODEL_NAME = "ai-forever/ruT5-base"
def load_tokenizer(model_name: str = DEFAULT_MODEL_NAME):
    return AutoTokenizer.from_pretrained(model_name)
tokenizer = load_tokenizer(MODEL_NAME)


Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

ds DatasetDict({
    train: Dataset({
        features: ['date', 'url', 'summary', 'title', 'text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['date', 'url', 'summary', 'title', 'text'],
        num_rows: 200
    })
})


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
raw_dataset['train'][0]

{'date': datetime.datetime(2010, 6, 1, 10, 35, 49),
 'url': 'https://www.gazeta.ru/auto/2010/05/31_a_3377717.shtml',
 'summary': 'С 2011 года правительство отменяет самый раздражающий граждан налог – транспортный. Но поборы автомобилистов не прекратятся – налоги завуалируют в бензиновые акцизы и платные дороги, а цены на товары подскочат. Зато теперь собираемые деньги обещают пустить только на строительство и содержание дорог.',
 'title': 'Налог в бак',
 'text': 'Сегодня транспортный налог начисляется в зависимости от мощности автомобиля, причем цена для «сильных» машин выше, чем для малолитражек. Также ставку налога могут корректировать региональные власти: согласно Налоговому кодексу, базовый тариф, установленный правительством, может быть уменьшен в пять раз или увеличен до 10 раз. Сборы идут в региональные бюджеты, откуда растекаются на общие нужды. Транспортный налог — один из основных источников бюджетных доходов — предлагается направить исключительно на дорожные фонды. Так, авто

## задачи


In [5]:
@dataclass
class TaskConfig:
    name: str
    target_field: str
    prefix: str
    max_target_length: int
    num_train_epochs: float = 3.0
    learning_rate: float = 3e-4
    batch_size: int = 4
    generation_max_length: int = 64
    num_beams: int = 4

tasks = [
    TaskConfig(
        name="summary",
        target_field="summary",
        prefix="summarize: ",
        max_target_length=96,
        generation_max_length=96,
        num_train_epochs=3,
        batch_size=2,
    ),
    TaskConfig(
        name="title",
        target_field="title",
        prefix="title: ",
        max_target_length=32,
        generation_max_length=32,
        num_train_epochs=3,
        batch_size=2,
    ),
]


## train


In [6]:
metrics = {}
for task in tasks:
    print("train", task.name)
    _, eval_metrics = train_task(
        raw_dataset=raw_dataset,
        tokenizer=tokenizer,
        task=task,
        output_root=OUTPUT_ROOT,
        model_name=MODEL_NAME,
    )
    metrics[task.name] = eval_metrics
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

metrics


train summary


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/il_dimas/Programming_Projects/NLP_LABS/NLP_LAB_5/lab5_utils.py:184: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


  0%|          | 0/1500 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'loss': 6.3835, 'grad_norm': 26.755495071411133, 'learning_rate': 8.6e-05, 'epoch': 0.1}
{'loss': 3.819, 'grad_norm': 11.336626052856445, 'learning_rate': 0.000186, 'epoch': 0.2}
{'loss': 3.6038, 'grad_norm': 30.85675811767578, 'learning_rate': 0.00028599999999999996, 'epoch': 0.3}
{'loss': 3.8881, 'grad_norm': 9.423216819763184, 'learning_rate': 0.0002904444444444444, 'epoch': 0.4}
{'loss': 3.7614, 'grad_norm': 10.112654685974121, 'learning_rate': 0.0002793333333333333, 'epoch': 0.5}
{'loss': 3.639, 'grad_norm': 7.924348831176758, 'learning_rate': 0.00026822222222222216, 'epoch': 0.6}
{'loss': 3.5565, 'grad_norm': 7.1118083000183105, 'learning_rate': 0.0002571111111111111, 'epoch': 0.7}
{'loss': 3.5211, 'grad_norm': 7.248687267303467, 'learning_rate': 0.00024599999999999996, 'epoch': 0.8}
{'loss': 3.4715, 'grad_norm': 6.208219528198242, 'learning_rate': 0.00023488888888888886, 'epoch': 0.9}
{'loss': 3.2028, 'grad_norm': 6.910845756530762, 'learning_rate': 0.00022377777777777774, 'epo

...

{'train_runtime': 213.5656, 'train_samples_per_second': 14.047, 'train_steps_per_second': 7.024, 'train_loss': 3.457900324503581, 'epoch': 3.0}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


  0%|          | 0/100 [00:00<?, ?it/s]

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'summary': {'eval_loss': 2.7527894973754883,
  'eval_rouge1': 0.1721,
  'eval_rouge2': 0.0407,
  'eval_rougeL': 0.1666,
  'eval_rougeLsum': 0.1665,
  'eval_gen_len': 51.615,
  'eval_runtime': 60.9136,
  'eval_samples_per_second': 3.283,
  'eval_steps_per_second': 1.642,
  'epoch': 3.0},
 'title': {'eval_loss': 4.710695266723633,
  'eval_rouge1': 0.0187,
  'eval_rouge2': 0.0,
  'eval_rougeL': 0.0187,
  'eval_rougeLsum': 0.0187,
  'eval_gen_len': 7.715,
  'eval_runtime': 13.8251,
  'eval_samples_per_second': 14.466,
  'eval_steps_per_second': 7.233,
  'epoch': 3.0}}

## тест


In [7]:
sample_text = raw_dataset["test"][10]["text"]

generated_summary = generate_sample(
    model_path=OUTPUT_ROOT / "summary",
    prefix="summarize: ",
    text=sample_text,
    max_length=96,
)

generated_title = generate_sample(
    model_path=OUTPUT_ROOT / "title",
    prefix="title: ",
    text=sample_text,
    max_length=32,
)

print("src", sample_text[:400])
print("title", generated_title)
print("sum", generated_summary)


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


src Ожирение значительно повышает риск тяжелого течения COVID-19, предупреждают врачи из Лилльского университета во Франции — пациенты с лишним весом чаще оказываются в отделениях интенсивной терапии и нуждаются в искусственной вентиляции легких. Об этом они рассказали на ежегодном Европейском и международном конгрессе по вопросам ожирения, который в этом году прошел онлайн. В начале апреля в Лилльско
title Ожирение не дожил до конца
sum В тяжелых случаях заболевания COVID-19 пациенты с избыточным весом чаще оказываются в отделениях интенсивной терапии. Ожирение значительно повышает риск тяжелого течения COVID-19, предупреждают эксперты.
